In [11]:
import json
from glob import glob
from tqdm import tqdm
import random
import os 
from shutil import copyfile
from tqdm.auto import tqdm
import multiprocessing
from functools import partial
from contextlib import contextmanager
classes = [
    "사람",
    "오토바이탄사람",
    "자전거탄사람",
    "킥보드탄사람",
    "오토바이",
    "자전거",
    "킥보드",
]


In [3]:
image_paths = glob('D:/data/01.데이터/2.Validation/원천데이터/VS_사건사고데이터이미지/사건사고데이터이미지/*/*/*.jpg')

In [4]:
random.shuffle(image_paths)

In [5]:
len(image_paths)

94658

In [6]:
train_paths = image_paths[:50000] 
val_paths = image_paths[50000:52000] 
test_paths = image_paths[52000:54000]

In [7]:
print(len(train_paths))

50000


In [8]:
save_image_dir = 'C:/Users/Navy/Desktop/maicon/Yolov5_StrongSORT_OSNet/yolov5/dataset/images/'
save_label_dir = 'C:/Users/Navy/Desktop/maicon/Yolov5_StrongSORT_OSNet/yolov5/dataset/labels/'
origin_label_dir = 'D:/data/01.데이터/2.Validation/라벨링데이터/사건사고데이터이미지/'

In [9]:
save_image_dir_linux = '/mnt/c/Users/Navy/Desktop/maicon/Yolov5_StrongSORT_OSNet/yolov5/dataset/images/'

In [10]:
def convert(paths,txt_path):
    f_txt = open(txt_path,'w', encoding='UTF-8')

    for path in tqdm(paths):
        filename = os.path.basename(path)
        f_txt.write(save_image_dir_linux+filename+"\n")

    f_txt.close()

convert(train_paths,'C:/Users/Navy/Desktop/maicon/Yolov5_StrongSORT_OSNet/yolov5/dataset/train.txt')
convert(val_paths,'C:/Users/Navy/Desktop/maicon/Yolov5_StrongSORT_OSNet/yolov5/dataset/val.txt')
convert(test_paths,'C:/Users/Navy/Desktop/maicon/Yolov5_StrongSORT_OSNet/yolov5/dataset/test.txt')

  0%|          | 0/50000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

In [15]:
def convert(paths):

    for path in tqdm(paths):
        filename = os.path.basename(path)
        label_path = origin_label_dir+'/'+'/'.join(path.split('\\')[-3:]).replace('.jpg','.json')
        try:
            with open(label_path,'r', encoding='UTF-8') as f:
                json_object = json.load(f)
        except:
            print(path)
            continue
        
        size = json_object["image"]["size"]
        label_data = ""
        for ano in json_object['annotation']:
                class_name = classes.index(ano["property"]["name"])
                bbox = ano["bndbox"]
                x = (bbox['xmin']+bbox['xmax'])/(2*size['width'])
                y = (bbox['ymin']+bbox['ymax'])/(2*size['height'])
                width = (bbox['xmax'] - bbox['xmin'])/size['width']
                height = (bbox['ymax'] - bbox['ymin'])/size['height']
                label_data = label_data + str(class_name) + " " + str(x) + " " + str(y) + " " + str(width) + " " + str(height) + "\n"

        f_label = open(save_label_dir+filename.replace('.jpg','.txt'),'w', encoding='UTF-8')
        f_label.write(label_data)
        f_label.close()
        
        result_path = save_image_dir+filename
        copyfile(path,result_path)

print("load")
p1 = Process(target=convert, args=(train_paths[:10000],))
p2 = Process(target=convert, args=(train_paths[10000:20000],))
p3 = Process(target=convert, args=(train_paths[20000:30000],))
p4 = Process(target=convert, args=(train_paths[30000:40000],))
p5 = Process(target=convert, args=(train_paths[40000:50000],))
p6 = Process(target=convert, args=(val_paths,))
p7 = Process(target=convert, args=(test_paths,))
p1.start();p2.start();p3.start();p4.start();p5.start();p6.start();p7.start();
print("start")
p1.join();p2.join();p3.join();p4.join();p5.join();p6.join();p7.join();
print("end")
# p2 = Process(target=f, args=("proc_2",))
# convert(train_paths)
# convert(val_paths)
# convert(test_paths)

In [15]:
import time
from multiprocessing import Pool

def count(process_name):
    for i in range(1, 100001):
        print(process_name, i)

start_time = time.time()
p_list = ["proc_1", "proc_2", "proc_3", "proc_4"]
pool = Pool(processes = 4)
pool.map(count, p_list)
pool.close()
pool.join()

print(time.time() - start_time)